In [13]:
import json

with open('/Users/sunnyyu/Desktop/Children_Speech_Books_NLP/data/data_with_all_info/MERGED_DATA_SPEECH.json', 'r') as file:
    generics_data = json.load(file)

In [20]:
# import the classified file with 1-5 feature categories

with open('/Users/sunnyyu/Desktop/Children_Speech_Books_NLP/code/Python/analysis_files/GENERIC_FEATURE_SPEECH.json', 'r') as file:
    features = json.load(file)

In [11]:
sentence_map = {}
for book in generics_data:
    for entry in book:
        sentence = entry['sentence']
        tags = entry.get('tags', [])
        subjects = entry.get('subjects', [])
        age_min = entry['age_min']
        age_max = entry['age_max']
        sentence_map[sentence] = {'tags': tags, 'subjects': subjects, 'age_min': age_min, 'age_max': age_max}

# Combine data
combined_data2 = []
for feature in features:
    sentence = feature['sentence']
    classification = feature['classification']
    
    # Preprocess classification to convert it to a list of integers
    if classification:
        classification_list = [int(x) for x in classification.split(',')]  # Convert string to list of integers

    # Find the corresponding tags and subjects
    if sentence in sentence_map:
        combined_entry = {
            'sentence': sentence,
            'tags': sentence_map[sentence]['tags'],
            'subjects': sentence_map[sentence]['subjects'],
            'classification': classification_list,  # Store as list of integers
            'age_min': sentence_map[sentence]['age_min'],
            'age_max': sentence_map[sentence]['age_max'],
            "category": "GENERIC"
        }
        combined_data2.append(combined_entry)

ValueError: invalid literal for int() with base 10: 'خمسة'

In [21]:
sentence_map = {}
for entry in generics_data:
        sentence = entry['sentence']
        tags = entry.get('tags', [])
        subjects = entry.get('subjects', [])
        age_min = entry['age_min'],
        age_max = entry['age_max'],
        sentence_map[sentence] = {'tags': tags, 'subjects': subjects, 'age_min': age_min, 'age_max': age_max}

# Combine data
combined_data3 = []
for feature in features:
    sentence = feature['sentence']
    classification = feature['classification']
    if classification:
    # Convert each value to an integer, skip if it cannot be converted
        classification_list = []
        for item in classification.split(','):
            try:
                classification_list.append(int(item))  # Attempt to convert to int
            except ValueError:
                pass  # If it cannot be converted, ignore it (do nothing)



    # Find the corresponding tags and subjects
    if sentence in sentence_map:
        combined_entry = {
            'sentence': sentence,
            'tags': sentence_map[sentence]['tags'],
            'subjects': sentence_map[sentence]['subjects'],
            'classification': classification_list,
            'age_min': sentence_map[sentence]['age_min'],
            'age_max': sentence_map[sentence]['age_max'],
             "category": "GENERIC"
        }
        combined_data3.append(combined_entry)

In [23]:
len(combined_data)

820

In [9]:
len(combined_data1)

750

In [19]:
len(combined_data2)

5547

In [22]:
len(combined_data3)

6329

In [24]:
final = combined_data + combined_data1 + combined_data2 + combined_data3
len(final)

13446

In [25]:
final

[{'sentence': 'Every day, Bailey Beaver eats ice cream for breakfast, candy for lunch and cake for dinner.',
  'tags': [{'entity': 'Non-Living Kind',
    'score': 0.9749864339828491,
    'index': 2,
    'word': 'day',
    'start': 6,
    'end': 9},
   {'entity': 'Biological Kind',
    'score': 0.9390778541564941,
    'index': 4,
    'word': 'bailey beaver',
    'start': 11,
    'end': 24},
   {'entity': 'Behavioral',
    'score': 0.9895424842834473,
    'index': 6,
    'word': 'eats',
    'start': 25,
    'end': 29},
   {'entity': 'Artifacts',
    'score': 0.6457382440567017,
    'index': 7,
    'word': 'ice cream',
    'start': 30,
    'end': 39},
   {'entity': 'Artifacts',
    'score': 0.6778071522712708,
    'index': 10,
    'word': 'breakfast',
    'start': 44,
    'end': 53},
   {'entity': 'Artifacts',
    'score': 0.7634249329566956,
    'index': 12,
    'word': 'candy',
    'start': 55,
    'end': 60},
   {'entity': 'Artifacts',
    'score': 0.6542389392852783,
    'index': 14,


In [29]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder



# Prepare the dataset for logistic regression
data = []
excluded_pronouns = {'he', 'you', 'we', 'she', 'i', 'they', 'him', 'her', 'them', 'his', 'hers', 'theirs', 'it'}

for item in final:
    tags = item.get('tags', [])
    subjects = item.get('subjects', [])
    classification = item.get('classification', [])
    category = item.get('category', None)

    for subject_info in subjects:
        subject = subject_info['subject'].lower()
        if subject not in excluded_pronouns:
            for tag in tags:
                if subject == tag['word'].lower():
                    entity = tag['entity']
                    # Add data entry
                    data.append({
                        'entity_type': entity,
                        'classification': classification[0] if classification else None,
                        'category': 1 if category == 'GENERIC' else 0  # Binary outcome: 1 = GENERIC, 0 = HABITUAL
                    })

# Convert to DataFrame
df = pd.DataFrame(data)

# One-hot encode entity_type and classification
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_features = encoder.fit_transform(df[['entity_type', 'classification']])
feature_names = encoder.get_feature_names_out(['entity_type', 'classification'])

# Combine encoded features with the target variable
X = pd.DataFrame(encoded_features, columns=feature_names)
y = df['category']

# Add a constant for the intercept
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X).fit()

# Print the summary with coefficients, p-values, and odds ratios
print(model.summary())

# Exponentiate coefficients to interpret as odds ratios
odds_ratios = pd.DataFrame({
    'Predictor': ['Intercept'] + list(feature_names),
    'Odds Ratio': [sm.tools.tools.exp(model.params[0])] + list(sm.tools.tools.exp(model.params[1:]))
})
print("\nOdds Ratios:\n", odds_ratios)


         Current function value: 0.461338
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:               category   No. Observations:                 6152
Model:                          Logit   Df Residuals:                     6137
Method:                           MLE   Df Model:                           14
Date:                Mon, 30 Dec 2024   Pseudo R-squ.:                  0.1955
Time:                        11:21:56   Log-Likelihood:                -2838.2
converged:                      False   LL-Null:                       -3528.0
Covariance Type:            nonrobust   LLR p-value:                3.676e-286
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                           -16.4735   8032.797     -0.002      0.998   -1.58e+04    1.57e+04
entity_t

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


AttributeError: module 'statsmodels.tools.tools' has no attribute 'exp'